# Sistema Multi-Agente de Consulta de Normativas Aeronáuticas

## Descripción

En este ejercicio, diseñarás un **sistema multi-agente que orquesta consultas a diferentes normativas aeronáuticas**. Un agente orquestador decidirá qué agentes especializados invocar según la consulta del usuario, y siempre verificará con la normativa EASA antes de dar una respuesta final.

### Arquitectura
```
Consulta del usuario
    ↓
[Agente Orquestador]
    ├→ [EASA] → Siempre consultado (obligatorio)
    ├→ [DEF-STAN] → Consultado si se menciona
    ├→ [EDA] → Consultado si se menciona
    ├→ [FAA] → Consultado si se menciona
    └→ [JSSG] → Consultado si se menciona
    ↓
Respuesta integral verificada con EASA
```

### Agentes Especializados Disponibles
- **DEF-STAN**: Normativa DEF STAN
- **EDA**: Normativa EDA (European Defence Agency)
- **FAA**: Normativa FAA (Federal Aviation Administration)
- **JSSG**: Normativa JSSG (Joint Service Specification Guide)
- **EASA**: Normativa EASA (European Union Aviation Safety Agency) - **Verificación obligatoria**

### Conceptos Clave
- **Connected Agent Tools**: Permite que el orquestador invoque agentes especializados
- **Arquitectura Multi-agente**: Consulta inteligente basada en el contexto de la pregunta
- **Verificación EASA**: Todas las respuestas deben ser verificadas con la normativa EASA

### Cargar librerías y variables de entorno

In [1]:
import os
from dotenv import load_dotenv, find_dotenv

from azure.ai.agents import AgentsClient
from azure.ai.agents.models import ConnectedAgentTool, MessageRole, ListSortOrder
from azure.identity import DefaultAzureCredential

load_dotenv(find_dotenv(usecwd=True))
project_endpoint = os.getenv("PROJECT_ENDPOINT")
model_deployment = os.getenv("MODEL_DEPLOYMENT_NAME")

### Conectar al Cliente de Agentes (Agent Client)

In [2]:
print("Connecting to Azure AI Agents Client...")
agents_client = AgentsClient(
    endpoint=project_endpoint,
    credential=DefaultAzureCredential(
        exclude_environment_credential=True,
        exclude_managed_identity_credential=True,
    )
)
print("Agents client connected successfully")

Connecting to Azure AI Agents Client...


Agents client connected successfully


### Configurar Agentes Especializados Existentes

Los agentes especializados ya están creados con conocimiento exclusivo de diferentes normativas. Configuramos las herramientas para conectar con ellos.

In [3]:
# IDs de los agentes especializados ya creados
AGENT_IDS = {
    "DEF-STAN": "asst_rkE8wccEYNW8VMkyYvGbCJGA",
    "EDA": "asst_fOW4towcWWIuHUN7Q8cBI05x",
    "FAA": "asst_acdzvNNQe6951L1v1wnIWRJJ",
    "JSSG": "asst_8w48R9lpBhAP5DluxYpYzMjE",
    "EASA": "asst_4Fh3Cj4SBLhjWmgeQLgXTzjV"
}

print("Configurando herramientas para agentes especializados...")
print(f"\nAgentes disponibles:")
for name, agent_id in AGENT_IDS.items():
    print(f"  - {name}: {agent_id}")

Configurando herramientas para agentes especializados...

Agentes disponibles:
  - DEF-STAN: asst_rkE8wccEYNW8VMkyYvGbCJGA
  - EDA: asst_fOW4towcWWIuHUN7Q8cBI05x
  - FAA: asst_acdzvNNQe6951L1v1wnIWRJJ
  - JSSG: asst_8w48R9lpBhAP5DluxYpYzMjE
  - EASA: asst_4Fh3Cj4SBLhjWmgeQLgXTzjV


### Crear Herramientas de Agentes Conectados (Connected Agent Tools)

Creamos herramientas para que el agente orquestador pueda invocar a cada uno de los agentes especializados.

In [4]:
print("Creando herramientas de agentes conectados...")

# Herramienta para DEF-STAN
defstan_tool = ConnectedAgentTool(
    id=AGENT_IDS["DEF-STAN"],
    name="DEF_STAN",
    description="Consulta la normativa DEF STAN. Usa esta herramienta cuando el usuario mencione específicamente DEF STAN o estándares de defensa británicos."
)

# Herramienta para EDA
eda_tool = ConnectedAgentTool(
    id=AGENT_IDS["EDA"],
    name="EDA",
    description="Consulta la normativa EDA (European Defence Agency). Usa esta herramienta cuando el usuario mencione específicamente EDA o normativa de defensa europea."
)

# Herramienta para FAA
faa_tool = ConnectedAgentTool(
    id=AGENT_IDS["FAA"],
    name="FAA",
    description="Consulta la normativa FAA (Federal Aviation Administration). Usa esta herramienta cuando el usuario mencione específicamente FAA o regulaciones estadounidenses."
)

# Herramienta para JSSG
jssg_tool = ConnectedAgentTool(
    id=AGENT_IDS["JSSG"],
    name="JSSG",
    description="Consulta la normativa JSSG (Joint Service Specification Guide). Usa esta herramienta cuando el usuario mencione específicamente JSSG o especificaciones militares."
)

# Herramienta para EASA (OBLIGATORIA)
easa_tool = ConnectedAgentTool(
    id=AGENT_IDS["EASA"],
    name="EASA",
    description="Consulta la normativa EASA (European Union Aviation Safety Agency). ESTA HERRAMIENTA DEBE SER SIEMPRE CONSULTADA antes de proporcionar una respuesta final al usuario."
)

print("Herramientas de agentes conectados creadas:")
print("  - DEF-STAN")
print("  - EDA")
print("  - FAA")
print("  - JSSG")
print("  - EASA (verificación obligatoria)")

Creando herramientas de agentes conectados...
Herramientas de agentes conectados creadas:
  - DEF-STAN
  - EDA
  - FAA
  - JSSG
  - EASA (verificación obligatoria)


### Crear el Agente Orquestador

Este agente analizará la consulta del usuario, decidirá qué agentes especializados invocar, y siempre verificará la respuesta con EASA.

In [5]:
orchestrator_agent_name = "normativas-orchestrator"
orchestrator_agent_instructions = """
Eres un agente orquestador especializado en normativas aeronáuticas. Tu función es:

1. Análisis de la Consulta: 
   - Analiza cuidadosamente la pregunta del usuario para identificar qué normativas específicas se mencionan o se requieren.
   
2. Selección de Agentes:
   - Si el usuario menciona DEF-STAN, DEF STAN o estándares de defensa británicos: consulta al agente DEF-STAN
   - Si el usuario menciona EDA o defensa europea: consulta al agente EDA
   - Si el usuario menciona FAA o regulaciones estadounidenses: consulta al agente FAA
   - Si el usuario menciona JSSG o especificaciones militares: consulta al agente JSSG
   
3. Verificación OBLIGATORIA con EASA:
   - SIEMPRE debes consultar al agente EASA antes de proporcionar tu respuesta final
   - La verificación con EASA es obligatoria independientemente de las otras normativas consultadas
   - Usa la información de EASA para validar, complementar o contrastar con las otras normativas

4. Respuesta Estructurada:
   - Proporciona una respuesta clara y estructurada
   - Indica claramente qué normativas has consultado
   - Incluye la verificación de EASA al final
   - Si hay conflictos entre normativas, señálalos claramente
   
5. Principios:
   - Sé preciso y técnico en tus respuestas
   - Cita las fuentes (normativas) consultadas
   - Si no estás seguro, indícalo claramente
   - Prioriza la seguridad y el cumplimiento normativo

Recuerda: La consulta a EASA es OBLIGATORIA en todas las respuestas.
"""

print("Creando agente orquestador...")
orchestrator_agent = agents_client.create_agent(
    model=model_deployment,
    name=orchestrator_agent_name,
    instructions=orchestrator_agent_instructions,
    tools=[
        defstan_tool.definitions[0],
        eda_tool.definitions[0],
        faa_tool.definitions[0],
        jssg_tool.definitions[0],
        easa_tool.definitions[0],
    ],
)
print(f"Agente orquestador creado (id: {orchestrator_agent.id})")
print(f"\nEl agente tiene acceso a {len(orchestrator_agent.tools)} herramientas:")
print("  - DEF-STAN")
print("  - EDA")
print("  - FAA")
print("  - JSSG")
print("  - EASA (verificación obligatoria)")

Creando agente orquestador...


Agente orquestador creado (id: asst_3t1rpqp9WOnWiy8whMRLob50)

El agente tiene acceso a 5 herramientas:
  - DEF-STAN
  - EDA
  - FAA
  - JSSG
  - EASA (verificación obligatoria)


### Realizar Consultas al Sistema Multi-Agente

Introduce tu consulta sobre normativas aeronáuticas en el `prompt`. El agente orquestador decidirá qué agentes consultar y siempre verificará con EASA.

In [6]:
print("Creando hilo de conversación...")
thread = agents_client.threads.create()
print(f"Hilo creado (id: {thread.id})")

# Introduce tu consulta aquí
prompt = "¿Qué requisitos establece la normativa FAA para la certificación de motores turbofan?"

print(f"\n{'='*80}")
print(f"CONSULTA DEL USUARIO:")
print(f"{'='*80}")
print(f"{prompt}")
print(f"{'='*80}\n")

print("Enviando consulta al agente orquestador...")
message = agents_client.messages.create(
    thread_id=thread.id,
    role=MessageRole.USER,
    content=prompt,
)

print("Procesando consulta... El agente orquestador está consultando las normativas necesarias.")
print("Esto puede tomar unos momentos...\n")

run = agents_client.runs.create_and_process(
    thread_id=thread.id, 
    agent_id=orchestrator_agent.id
)

print(f"Procesamiento completado con estado: {run.status}\n")

if run.status == "failed":
    print(f"❌ Error en la ejecución: {run.last_error}")
else:
    print(f"{'='*80}")
    print(f"RESPUESTA DEL AGENTE:")
    print(f"{'='*80}\n")
    
    messages = agents_client.messages.list(
        thread_id=thread.id, 
        order=ListSortOrder.ASCENDING
    )
    
    for message in messages:
        if message.text_messages:
            for text_msg in message.text_messages:
                if message.role != MessageRole.USER:
                    print(f"\n{text_msg.text.value}\n")
    
    print(f"{'='*80}")

Creando hilo de conversación...
Hilo creado (id: thread_bNlQvmZWN37VMXBHSOC6h6mA)

CONSULTA DEL USUARIO:
¿Qué requisitos establece la normativa FAA para la certificación de motores turbofan?

Enviando consulta al agente orquestador...
Procesando consulta... El agente orquestador está consultando las normativas necesarias.
Esto puede tomar unos momentos...

Procesamiento completado con estado: RunStatus.COMPLETED

RESPUESTA DEL AGENTE:


He consultado las normativas de la FAA y de EASA para responder a su consulta sobre los requisitos para la certificación de motores turbofan.

1. Requisitos FAA (14 CFR Parte 33):  
   - Esta normativa establece estándares detallados de diseño, materiales, protección contra incendio, sistemas de enfriamiento, y limitaciones de montaje.  
   - Incluye requisitos de seguridad específicos para turbofans, como protección contra formación de hielo e ingesta de objetos.  
   - Se exigen pruebas de ingesta de aves para validar seguridad ante impactos, así como